In [1]:
! pip install -qU langgraph==0.2.14 langchain==0.2.14 langchain_openai==0.1.23 langchain_core==0.2.35 langchain-community

In [5]:
from dotenv import load_dotenv
import os
import getpass
import openai

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode

In [7]:
load_dotenv()

True

In [9]:

openai_chat_model=ChatOpenAI(model='gpt-4o-mini')

In [13]:

# Setup tools
tavily_tool = TavilySearchResults(max_results=5)
tool_belt = [tavily_tool, ArxivQueryRun()]

# Model with tools function
def openai_chat_model_with_tools(input_data):
    model = openai_chat_model.bind_tools(tool_belt)
    if isinstance(input_data, dict):
        message = [HumanMessage(content=input_data.get("question", ""))]
    else:
        message = [HumanMessage(content=str(input_data))]
    response = model.invoke(message)
    return response

# Tool routing function
def route_to_tools(ai_message):
    if isinstance(ai_message, AIMessage):
        tool_response = ToolNode(tool_belt).invoke({"messages": [ai_message]})
        return tool_response.get("messages", [])
    return []

# Initial research prompt (for tool queries)
tool_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a research assistant. Use the available tools to find information.
    1. Use arxiv for academic papers
    2. Use tavily for supplementary context"""),
    ("human", "{question}")
])

# Final synthesis prompt
research_prompt = ChatPromptTemplate.from_messages([
    ("system", """Synthesize the information from the tools and answer the question.
    Include citations and structure your response clearly."""),
    ("human", """Question: {question}
    Context: {context}""")
])

# Chain construction
def prepare_final_input(input_dict):
    return {
        "question": input_dict["question"],
        "context": input_dict.get("context", "No context available")
    }

rag_chain_with_dynamic_access = (
    {
        "context": itemgetter("question") 
        | tool_prompt 
        | openai_chat_model_with_tools 
        | route_to_tools,
        "question": itemgetter("question")
    }
    | RunnableLambda(prepare_final_input)
    | research_prompt 
    | openai_chat_model_with_tools
    | StrOutputParser()
)

In [16]:
pwd

'/home/sahane/JasminLLC'

In [14]:
# Test the chain
response = rag_chain_with_dynamic_access.invoke({
    "question": "What does the 'context' in 'long context' refer to?"
})

In [15]:
from pprint import PrettyPrinter
# Set width to fit your screen
pp = PrettyPrinter(width=200, indent=2)

pp.pprint(response)

('The term "context" in "long context" refers to the amount of information that a model can process and remember in a single prompt. Long-context models are designed to handle significantly larger '
 'inputs compared to traditional models, allowing them to process hundreds of thousands to millions of tokens at once. This capability enables these models to analyze entire documents, books, or '
 'large datasets effectively.\n'
 '\n'
 '1. **Definition of Long Context**: Long context means that the model can retain and utilize a substantial amount of information simultaneously. This is crucial for tasks that require comprehensive '
 'understanding or analysis of extensive texts without losing relevant details (Source: [ZDNet](https://www.zdnet.com/article/what-does-a-long-context-window-mean-for-an-ai-model-like-gemini/)).\n'
 '\n'
 '2. **Enhanced Processing Capabilities**: Long-context language models simplify the analysis process by reducing the need for complex information retrieval tec